In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("./dataset/east_asia_population.csv")
df.dropna(inplace=True)

In [9]:
def make_age_code(col):
    return col.split(".")[2]

def make_age_col(code):
    _from = int(code[:2])
    _to = code[2:4]
    if (_to != "UP"):
        _to = int(_to)
    else:
        _to = ""
    
    return f"{_from}-{_to}"
    
df.columns = df.columns[:4].to_list() + df.columns[4:].map(lambda x: x.split(" ")[0]).to_list()

df["Series Code"] = df["Series Code"].apply(make_age_code)
df["Age"] = df["Series Code"].apply(make_age_col)

In [64]:
# groupby country, series code

def make_df(_from, _to=None):
    if _to == None:
        _to = _from;
    years = [str(year) for year in list(range(_from, _to + 1))]
    year_set = dict()
    for year in years:
        year_set = year_set | { year: "sum" }

    idx_cols = ["Country Code", "Series Code"];
    cols = idx_cols + ["Country Name", "Age"] + years
    return df[cols].groupby(["Country Code", "Age"], as_index=False)\
        .agg({ "Country Name": "first", "Series Code": "first" } | year_set)\
        .sort_values(idx_cols)\
        .drop(columns=idx_cols)

make_df(2023, 2024)

,Age,Country Name,2023,2024
0,0-4,China,60553055.0,56192096.0
9,5-9,China,87858748.0,84573993.0
1,10-14,China,88613542.0,89264348.0
2,15-19,China,81068710.0,82792864.0
3,20-24,China,78667103.0,78437481.0
...,...,...,...,...
97,60-64,Thailand,4947035.0,5012164.0
98,65-69,Thailand,4036404.0,4210052.0
99,70-74,Thailand,2941115.0,3076928.0
100,75-79,Thailand,1896461.0,2039750.0


In [71]:
make_df(2001, 2023).to_json("population.json", orient="records", indent=2)